In [ ]:
from google.colab import drive  #para interactuar con Google Drive desde un entorno Colab.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip uninstall numpy scipy scikit-learn -y  #reinstalación controlada de paquetes fundamentales para ciencia de datos
!pip install numpy==1.23.5 scipy==1.9.3 scikit-learn==1.2.2  #reinstalación controlada de paquetes fundamentales para ciencia de datos

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 137.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 130.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.9.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible

In [ ]:
import pandas as pd    # Para manipulación de datos estructurados (DataFrames)
import numpy as np      #Para operaciones numéricas eficientes
from sklearn.model_selection import train_test_split   #Divide los datos en conjuntos de entrenamiento y prueba
from sklearn.preprocessing import LabelEncoder         # Para codificar variables categóricas en numéricas
from sklearn.metrics import accuracy_score, confusion_matrix   #Calcula la precisión de las predicciones, Genera una matriz de confusión para evaluar el rendimiento
import matplotlib.pyplot as plt    # para Visualización de Datos
from scipy.stats import multivariate_normal  #Para trabajar con distribuciones normales multivariadas
from sklearn.datasets import make_classification  # Para crear datasets de clasificación sintéticos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report   #Calcula la precisión de las predicciones, Genera una matriz de confusión para evaluar el rendimiento

# Cargar el dataset desde el archivo subido
df = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# Mostrar columnas y detectar NaNs o infs
print("Columnas del dataset:", df.columns)
print("Cantidad de NaNs por columna:\n", df.isna().sum())

# Eliminar filas con NaNs o infinitos
df = df.replace([np.inf, -np.inf], np.nan)  # Reemplaza infinitos por NaN
df = df.dropna()  # Elimina cualquier fila con NaNs

# Separar variables independientes y la etiqueta
y = df['label']  # Cambiá 'label' si tu columna se llama diferente
X = df.drop(columns=['label'])

# Verificamos que no queden valores inválidos
assert not np.isnan(X.values).any(), "Todavía hay NaNs en X"
assert not np.isinf(X.values).any(), "Todavía hay infinitos en X"

# Verificamos las columnas
print("Columnas:", df.columns)

# Asegurarnos de que la columna objetivo se llama correctamente
# Por ejemplo, puede llamarse 'genero', 'Label', 'género', etc.
# Supongamos que es 'label', si no lo es cambiá esta línea:
y = df['label']  # CAMBIAR si es otra columna, por ej: df['genero']
X = df.drop(columns=['label'])

# Codificar las etiquetas (si son strings como 'hombre', 'mujer')
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X.values, y_encoded, test_size=0.2, random_state=42)


class GDA:
    def __init__(self):
        # Parámetros iniciales
        self.clases = None
        self.mu = {}
        self.sigma = {}
        self.priors = {}

    def fit(self, X, y):
        # Obtengo las clases únicas
        self.clases = np.unique(y)

        for k in self.clases:
            X_k = X[y == k]
            # Media por clase
            self.mu[k] = np.mean(X_k, axis=0)
            # Matriz de covarianza
            self.sigma[k] = np.cov(X_k, rowvar=False)
            self.sigma[k] += np.eye(self.sigma[k].shape[0]) * 1e-6
            # Probabilidad a priori
            self.priors[k] = len(X_k) / len(X)

    def predict(self, X):
        # Hago las predicciones
        res = []

        for x in X:
            probs = {}
            for k in self.clases:
                prob_conjunta = (1/np.sqrt(np.linalg.det(self.sigma[k]))) * np.exp(-0.5* np.dot(np.dot((x-self.mu[k]),np.linalg.inv(self.sigma[k])),(x-self.mu[k]).T))

                probs[k] = self.priors[k] * prob_conjunta

            res.append(max(probs, key=probs.get))

        return np.array(res)


# Entrenar y evaluar
gda = GDA()
gda.fit(X_train, y_train)
y_pred = gda.predict(X_test)

# Resultados
print("Exactitud en test:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))



# Exactitud (accuracy)
print("Exactitud (accuracy):", accuracy_score(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# Reporte de clasificación (precision, recall, f1-score)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Columnas del dataset: Index(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt',
       'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun',
       'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'mean_f0',
       'min_f0', 'max_f0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5',
       'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12',
       'mfcc_13', 'label'],
      dtype='object')
Cantidad de NaNs por columna:
 meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
mean_f0     2
min_f0      2
max_f0      2
mfcc_1      0
mfcc_2      0
mfcc_3      0
mfcc_4      0
mfcc_5      0
mfcc_6      0
mfcc_7      0
mfcc_8      0
mfcc_9      0
mfcc_10     0
mfcc_11     0
mfcc_12     0
mfcc_13     0
label       0
dtyp

In [ ]:
# Mostrar medias (mu) por clase
print("Media (mu) por clase:")
for k in gda.mu:
    print(f"Clase {k} ({le.inverse_transform([k])[0]}):")
    print(gda.mu[k])
    print()

# Mostrar matrices de covarianza (sigma) por clase
print("Matriz de covarianza (sigma) por clase:")
for k in gda.sigma:
    print(f"Clase {k} ({le.inverse_transform([k])[0]}):")
    print(gda.sigma[k])
    print()


Media (mu) por clase:
Clase 0 ('Female'):
[ 3.73651970e+03  4.33628190e-02 -3.66135226e-03 -1.70659655e-02
  7.11560431e-03  2.41815698e-02  6.22655074e-01  8.66210823e+00
  4.47349872e+03  1.20837176e-02  0.00000000e+00  3.73651970e+03
  3.27373515e-02  2.55827686e-03  1.25326572e-01  8.07915596e+03
  1.82562023e+03  1.47240782e+04  1.28984580e+04  4.92700054e-01
  1.51231654e+02  7.29639034e+01  3.47030634e+02 -3.82485979e+02
  1.13592083e+02  8.68207337e+00  2.86823905e+01  8.79296792e+00
  1.24616455e+01 -1.17190902e+01  1.24772706e+01 -2.36626997e+00
  2.24904238e+00 -4.69968855e+00  4.12503263e+00 -1.92955482e+00]

Clase 1 ('Male'):
[ 3.82273410e+03  5.51984958e-02 -3.63604857e-03 -1.98935801e-02
  9.55109549e-03  2.94446756e-02  6.14420313e-01  9.13818497e+00
  4.56237458e+03  1.42301809e-02  0.00000000e+00  3.82273410e+03
  4.12131579e-02  2.51914684e-03  1.52704277e-01  8.17378764e+03
  1.44749768e+03  1.51911041e+04  1.37436064e+04  5.60372249e-01
  3.66340805e+02  2.36470308

Conclusion:

El modelo LDA superó al QDA en accuracy (0.649 vs 0.597), demostrando mejor rendimiento. Al analizar las matrices de covarianza, se observa que el LDA presenta mayor estabilidad numérica, ya que promedia las matrices de ambas clases (principio de homocedasticidad), evitando el sobreajuste del QDA. Las matrices individuales del QDA (Female/Male) muestran diferencias menores al 10% en sus componentes principales (ej: [0,0] = 2.19e6 vs 2.10e6), lo que confirma que la covarianza compartida del LDA es válida. Esto, sumado a la simplicidad del modelo, hace que el QDA —más complejo y con peor generalización— no sea justificable en este caso. Por lo tanto, cuando las clases comparten estructura de covarianza, LDA es preferible por eficiencia y robustez.